In [ ]:
using AcausalNets
using QI
using LightGraphs
using LinearAlgebra

In [ ]:
import AcausalNets.Systems: 
    DiscreteSystem,
    multiply_star,
    divide_star,
    multiply_kron,
    sum_distribution,
    QuantumDistribution,
    shallowcopy,
    identity_distribution,
    permute_distribution,
    merge_systems

import AcausalNets.Inference:
    JoinTree,
    parent_cliques_dict

import AcausalNets.Common:
    Variable,
    is_subset

import AcausalNets.Algebra:
    event

In [ ]:
roA = Diagonal([.5, .5])
roBwA = Diagonal([.5, .5, .4, .6])
roCwA = Diagonal([.7, .3, .2, .8])
roDwB = Diagonal([.9, .1, .5, .5])
roEwC = Diagonal([.3, .7, .6, .4])
roFwDE = Diagonal([.01, .99, .01, .99, .01, .99, .99, .01])
roGwC = Diagonal([.8, .2, .1, .9])
roHwEG = Diagonal([.05, .95, .95, .05, .95, .05, .95, .05])

var_a = Variable(:a, 2)
var_b = Variable(:b, 2)
var_c = Variable(:c, 2)
var_d = Variable(:d, 2)
var_e = Variable(:e, 2)
var_f = Variable(:f, 2)
var_g = Variable(:g, 2)
var_h = Variable(:h, 2)

sys_a = DiscreteQuantumSystem([var_a], roA)
sys_b = DiscreteQuantumSystem([var_a], [var_b], roBwA)
sys_c = DiscreteQuantumSystem([var_a], [var_c], roCwA)
sys_d = DiscreteQuantumSystem([var_b], [var_d], roDwB)
sys_e = DiscreteQuantumSystem([var_c], [var_e], roEwC)
sys_f = DiscreteQuantumSystem([var_d, var_e], [var_f], roFwDE)
sys_g = DiscreteQuantumSystem([var_c], [var_g], roGwC)
sys_h = DiscreteQuantumSystem([var_e, var_g], [var_h], roHwEG)


example_an = AcausalNet()
push!(example_an, sys_a)
push!(example_an, sys_b)
push!(example_an, sys_c)
push!(example_an, sys_d)
push!(example_an, sys_e)
push!(example_an, sys_f)
push!(example_an, sys_g)
push!(example_an, sys_h)

# gplot(example_bn.dag, nodelabel=nodes(example_bn))
# [v.target for v in example_bn.cpds]

In [ ]:
# roAB =1/3*(ket(0,9)+ket(4,9)+ket(8,9))* (bra(0,9)+bra(4,9)+bra(8,9)) 

roAB = 1/6*(ket(0,9)+ket(1,9))*(bra(0,9)+bra(1,9))+
1/6*(ket(4,9)+ket(5,9))*(bra(4,9)+bra(5,9))+
1/6*(ket(8,9)+ket(6,9))*(bra(8,9)+bra(6,9))

roCwAB = Diagonal([
        0,1/2,1/2, #A=0, B=0
        0,0,1, #A=0, B=1
        0,1,0, #A=0, B=2
        0,0,1, #A=1, B=0
        1/2,0,1/2, #A=1, B=1
        1,0,0, #A=1, B=2
        0,1,0, #A=2, B=0
        1,0,0, #A=2, B=1
        1/2,1/2,0 #A=2, B=2
        ]); #

var_aq = Variable(:a, 3)

var_bq = Variable(:b, 3)
var_cq = Variable(:c, 3)

sys_ab = DiscreteQuantumSystem([var_aq, var_bq], roAB)
sys_c_ab = DiscreteQuantumSystem([var_aq, var_bq], [var_cq], roCwAB)
quantum_an = AcausalNet()

push!(quantum_an, sys_ab)
push!(quantum_an, sys_c_ab)

In [ ]:
roA = Diagonal([1/3, 1/3, 1/3])

roB = Diagonal([1/3, 1/3, 1/3])
roCwAB = Diagonal([
        0,1/2, 1/2, #A=0, B=0
        0,0,1, #A=0, B=1
        #CHANGED to that ro(C | AB) != ro(C | BA) - Originally BE 0,1,0
        0,1,0, #A=0, B=2 
        0,0,1, #A=1, B=0
        1/2,0,1/2, #A=1, B=1
        1,0,0, #A=1, B=2
        0,1,0, #A=2, B=0
        1,0,0, #A=2, B=1|
        1/2,1/2,0 #A=2, B=2
        ]); #

var_am = Variable(:a, 3)

var_bm = Variable(:b, 3)
var_cm = Variable(:c, 3)

sys_am = DiscreteQuantumSystem([var_am], roA)

sys_bm = DiscreteQuantumSystem([var_bm], roB)

sys_c_abm = DiscreteQuantumSystem([var_am, var_bm], [var_cm], roCwAB)
classic_anm = AcausalNet()

push!(classic_anm, sys_am)
push!(classic_anm, sys_bm)
push!(classic_anm, sys_c_abm)

In [ ]:
an = example_an

In [ ]:
import AcausalNets.Inference: moral_graph
mg = moral_graph(an)
println(variables_names(an))
mg.fadjlist

In [ ]:
import AcausalNets.Inference: triangulate
triangulated, cliques = triangulate(mg, an)

println(variables_names(an))
triangulated.fadjlist

In [ ]:
import AcausalNets.Inference: 
    JoinTree,
    Evidence,
    apply_evidence,
    apply_observations
jt = JoinTree(cliques, an);

In [ ]:
[(k, variables(v)) for (k, v) in jt.vertex_to_cluster]

In [ ]:
par_cli_dict = parent_cliques_dict(cliques, an)
a_obs = Evidence([var_aq], real(ketbra(0,0,3)))
b_obs = Evidence([var_bq], real(ketbra(2,2,3)))
c_obs = Evidence([var_cq], real(ketbra(1,1,3)))

observations = [b_obs, c_obs] #, e_obs]
# observations = DiscreteSystem{Array{Float64,2}}[]

In [ ]:
obs_jt = apply_observations(jt, par_cli_dict, observations)

In [ ]:
ptrace(real(obs_jt.vertex_to_cluster[1].distribution), [2,2,2], [3])

In [ ]:
import AcausalNets.Inference: single_message_pass

In [ ]:
passed = single_message_pass(4, 5, jt)
passed_2 = single_message_pass(5, 4, passed);

ptrace(distribution(passed_2.vertex_to_cluster[4]), [2,2,2], [3])

In [ ]:
ptrace(distribution(passed_2.vertex_to_cluster[5]), [2,2,2], [1])# bc in abc and bcd are now consistent

In [ ]:
import AcausalNets.Inference: global_propagation

propagated_jt = global_propagation(jt)

In [ ]:
ptrace(distribution(propagated_jt.vertex_to_cluster[4]), [2,2,2], [3])

In [ ]:
ptrace(distribution(propagated_jt.vertex_to_cluster[5]), [2,2,2], [3])

In [ ]:
real(roAB)

In [ ]:
dist = distribution(jt.vertex_to_cluster[1])
real(ptrace(dist, [3,3,3], [3]))